In [128]:
import pandas
import numpy
import torch

In [129]:
# components*components
# für reihe welche die komponente c_1 indiziert: gebe wahrscheinlichkeit, dass c_2 auch gefailt ist
a = torch.zeros((10,10))


In [130]:
b = torch.tensor([0,1,1,0,0,0,0,0,0,0])
b2 = torch.tensor([0,1,1,1,0,0,1,0,0,1])

In [131]:
c = torch.cartesian_prod(b.nonzero().flatten(), b.nonzero().flatten())

In [132]:
c

tensor([[1, 1],
        [1, 2],
        [2, 1],
        [2, 2]])

In [133]:
for index_tuple in torch.cartesian_prod(b.nonzero().flatten(), b.nonzero().flatten()):
    a[index_tuple[0], index_tuple[1]] += 1

In [134]:
a

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [135]:
torch.clamp_min(a.diag(), 1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [136]:
a.div(torch.clamp_min(a.diag(), 1)).t(), a.diag()/a.diag().sum()

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 tensor([0.0000, 0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]))

In [137]:
import seaborn as sns

In [138]:
a

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [139]:
list(string.ascii_letters[:10])

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

In [140]:
import string

import plotly.express as px
px.imshow(a.div(torch.clamp_min(a.diag(), 1)).t(), x=list(string.ascii_letters[:10]), y=list(string.ascii_letters[:10]), text_auto=True)

In [141]:
wandb.log({f'{shop_name} Conditional Probabilities': wandb.plots.HeatMap(Components.value_list(), Components.value_list(), a.div(torch.clamp_min(a.diag(), 1)).t(), show_text=True)})

NameError: name 'wandb' is not defined

In [ ]:

from numpy import float32
from torchmetrics import KLDivergence
import scipy.special as special

epsilon = torch.tensor(1e-8, dtype=torch.float64)
metric = KLDivergence(log_prob=False, reduction="sum")
metric = special.kl_div
distributions = [torch.tensor([0,0.1,0.8,0.1]), torch.tensor([0.1,0.15,0.7,0.05]), torch.tensor([0.1,0.1,0.1,0.7]), torch.tensor([0.5,0.5,0.0,0.0])]
for dist in distributions:
    num_nonzero = dist.count_nonzero()
    num_zeros = len(dist) - num_nonzero
    remove_from_nonzero = num_zeros * epsilon / num_nonzero
    dist.copy_(torch.where(dist != 0, dist - remove_from_nonzero, 0))
    dist.copy_(torch.where(dist == 0, epsilon, dist))
    print(dist)
    print(sum(dist))
    print()
num_buffers = len(distributions)
distances = torch.zeros((num_buffers, num_buffers))
for index_tuple in distances.new_ones(num_buffers, num_buffers).triu().nonzero():
    distances[index_tuple[0], index_tuple[1]] = torch.mean(metric(distributions[index_tuple[0]], distributions[index_tuple[1]]) + metric(distributions[index_tuple[1]], distributions[index_tuple[0]]))

tensor([1.0000e-08, 1.0000e-01, 8.0000e-01, 1.0000e-01])
tensor(1.)

tensor([0.1000, 0.1500, 0.7000, 0.0500])
tensor(1.)

tensor([0.1000, 0.1000, 0.1000, 0.7000])
tensor(1.)

tensor([5.0000e-01, 5.0000e-01, 1.0000e-08, 1.0000e-08])
tensor(1.)



In [ ]:
metric()

TypeError: kl_div() takes from 2 to 3 positional arguments but 0 were given

In [ ]:
1e-1 == 0.1

True

In [ ]:
distances

tensor([[0.0000, 0.4200, 1.0587, 6.4193],
        [0.0000, 0.0000, 0.7258, 3.6203],
        [0.0000, 0.0000, 0.0000, 3.8860],
        [0.0000, 0.0000, 0.0000, 0.0000]])

In [ ]:
a

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 6., 6., 3., 0., 0., 3., 0., 0., 3.],
        [0., 6., 6., 3., 0., 0., 3., 0., 0., 3.],
        [0., 3., 3., 3., 0., 0., 3., 0., 0., 3.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 3., 3., 3., 0., 0., 3., 0., 0., 3.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 3., 3., 3., 0., 0., 3., 0., 0., 3.]])

In [ ]:
a2 = a+1

In [ ]:
import torch.nn.functional as F

In [ ]:
torch.cdist(a, a2, p=2).sum()

tensor(600.1426)

torch.tensor()

In [ ]:
torch.pow(a-a2,2).sum().sqrt()

tensor(10.)

In [150]:
actions = torch.stack([torch.arange(10).mul(2), torch.arange(10).mul(3)])
actions
selected_action = torch.tensor([[4], [5]])

In [5]:
import math


runs_per_node_parallel = 30
nodes = 5
out = []
port = 50000
for off_policy_factor in [0.0, 0.25, 0.5, 0.75, 1.0]:
    for strategy in ["off", "replace", "combine", "knowledge_destillation"]:
        for use_cat_off_loss in [True, False]:
            for use_cat_retrain in [True, False]:
                for n_retrain in [1,20,100]:
                    for batch_size in [1,20,100]:
                        for sampling in [False, "positive", "balanced"]:
                            for use_exploration in [True, False]:
                                command=f"WANDB_CACHE_DIR=./wandb_cache WANDB_CONFIG_DIR=./wandb_config MRUBIS_PORT={port} python3 main.py --runner-master --on-off-policy --episodes 200"
                                command += f" --off-policy-factor {off_policy_factor} --transfer-strategy {strategy} --n-retrain {n_retrain} --batch-size {batch_size}"
                                if use_cat_off_loss:
                                    command += " --off-policy-loss-cat"
                                if use_cat_retrain:
                                    command += " --retrain-cat"
                                if sampling == "positive":
                                    command += " --positive-sampling"
                                if sampling == "balanced":
                                    command += " --balanced-sampling"
                                if use_exploration:
                                    command += " --use-exploration"
                                command += f" > mrubis{port}.log"
                                out.append(command)
                                port += 1
counter = 0
lines_per_file = math.ceil(len(out) / nodes / runs_per_node_parallel)
for start in range(0, len(out), lines_per_file):
    end = start  + lines_per_file
    with open(f"run_experiment_{counter}.bash", "w") as filehandle:
        filehandle.write(" && \n".join(out[start:end]))
    counter += 1
if end < len(out):
    with open(f"run_experiment_{counter}.bash", "w") as filehandle:
        filehandle.write(" && \n".join(out[end:len(out)]))
    counter += 1

node_index = 0
for start in range(0, counter, runs_per_node_parallel):
    end = min(counter, start + runs_per_node_parallel)
    with open(f"start_node_{node_index}.bash", "w") as filehandle:
        filehandle.write("&& sleep 1 &&".join([f"(bash run_experiment_{bash_num}.bash &)" for bash_num in range(start, end)]))
    node_index += 1

if end < counter:
    with open(f"start_node_{node_index}.bash", "w") as filehandle:
        filehandle.write("&& sleep 1 &&".join([f"(bash run_experiment_{bash_num}.bash &)" for bash_num in range(end, counter)]))
